In [3]:
import numpy as np
from numba import cuda, njit
import time
import math

# 1. GPU (CUDA) Kernel
@cuda.jit
def gpu_calc(vize, final, sonuclar):
    idx = cuda.grid(1)
    if idx < vize.size:
        sonuclar[idx] = (vize[idx] * 0.4) + (final[idx] * 0.6)

# 2. CPU (Seri İşlem) - Klasik for döngüsü
def cpu_calc_serial(vize, final, n):
    res = np.empty(n, dtype=np.float32)
    for i in range(n):
        res[i] = (vize[i] * 0.4) + (final[i] * 0.6)
    return res

def run_comparison():
    N = 100_000_000 # 100 Milyon Kayıt
    h_vize = np.random.randint(0, 101, size=N).astype(np.float32)
    h_final = np.random.randint(0, 101, size=N).astype(np.float32)

    # --- CPU HESAPLAMA ---
    start = time.time()
    cpu_res = cpu_calc_serial(h_vize, h_final, N)
    cpu_time = time.time() - start
    print(f"CPU (Seri) Süre: {cpu_time:.4f} saniye")

    # --- GPU HESAPLAMA ---
    # Veriyi GPU'ya taşı
    d_vize = cuda.to_device(h_vize)
    d_final = cuda.to_device(h_final)
    d_sonuclar = cuda.device_array(N, dtype=np.float32)

    threads = 256
    blocks = math.ceil(N / threads)

    start = time.time()
    gpu_calc[blocks, threads](d_vize, d_final, d_sonuclar)
    cuda.synchronize() # GPU'nun işini bitirmesini bekle
    gpu_time = time.time() - start
    
    # Sonuçları geri çek
    gpu_res = d_sonuclar.copy_to_host()
    print(f"GPU (CUDA) Süre: {gpu_time:.4f} saniye")

    print(f"\nSonuç: GPU, CPU'dan {cpu_time / gpu_time:.1f}x daha hızlı!")



In [4]:
run_comparison()

CPU (Seri) Süre: 35.6904 saniye
GPU (CUDA) Süre: 0.0518 saniye

Sonuç: GPU, CPU'dan 689.0x daha hızlı!


In [16]:
import numpy as np
from numba import cuda
import time
import math

# --- 1. YÖNTEM: CUDA KERNEL (GPU) ---
@cuda.jit
def osym_cuda_kernel(d_tr_d, d_tr_y, d_mat_d, d_mat_y, d_obp, d_sonuclar):
    idx = cuda.grid(1)
    if idx < d_sonuclar.size:
        # Net Hesaplama
        tr_net = d_tr_d[idx] - (d_tr_y[idx] * 0.25)
        mat_net = d_mat_d[idx] - (d_mat_y[idx] * 0.25)
        
        # Negatif net kontrolü
        if tr_net < 0: tr_net = 0
        if mat_net < 0: mat_net = 0
        
        # Puan Hesaplama (Formül: Taban + (Netler * 3.3) + OBP)
        d_sonuclar[idx] = 100 + (tr_net * 3.3) + (mat_net * 3.3) + (d_obp[idx] * 0.6)

# --- 2. YÖNTEM: SAF PYTHON DÖNGÜSÜ (CPU - Yavaş) ---
def cpu_loop_hesapla(tr_d, tr_y, mat_d, mat_y, obp, n):
    sonuclar = np.zeros(n, dtype=np.float32)
    for i in range(n):
        tr_net = tr_d[i] - (tr_y[i] * 0.25)
        mat_net = mat_d[i] - (mat_y[i] * 0.25)
        
        if tr_net < 0: tr_net = 0
        if mat_net < 0: mat_net = 0
        
        sonuclar[i] = 100 + (tr_net * 3.3) + (mat_net * 3.3) + (obp[i] * 0.6)
    return sonuclar

# --- 3. YÖNTEM: NUMPY (CPU - Optimize) ---
def cpu_numpy_hesapla(tr_d, tr_y, mat_d, mat_y, obp):
    # Vektörel işlem (Döngü yok, C seviyesinde işlem)
    tr_net = tr_d - (tr_y * 0.25)
    mat_net = mat_d - (mat_y * 0.25)
    
    # Clip ile negatifleri 0 yapma (NumPy yöntemi)
    tr_net = np.clip(tr_net, 0, None)
    mat_net = np.clip(mat_net, 0, None)
    
    return 100 + (tr_net * 3.3) + (mat_net * 3.3) + (obp * 0.6)

def main():
    # Veri Boyutu: 2 Milyon yerine bekleme süresini azaltmak için 
    # Python Loop testinde 100.000, diğerlerinde 5.000.000 yapalım.
    # Ancak adil kıyas için burada hepsini 1.000.000 (1 Milyon) yapıyoruz.
    N = 10_000_000 
    
    print(f"--- {N} ADAY İÇİN PERFORMANS TESTİ BAŞLIYOR ---\n")

    # Veri Üretimi
    h_tr_d = np.random.randint(0, 41, size=N).astype(np.float32)
    h_tr_y = np.random.randint(0, 10, size=N).astype(np.float32)
    h_mat_d = np.random.randint(0, 41, size=N).astype(np.float32)
    h_mat_y = np.random.randint(0, 10, size=N).astype(np.float32)
    h_obp = np.random.randint(50, 101, size=N).astype(np.float32)

    # ---------------------------------------------------------
    # TEST 1: SAF PYTHON LOOP (En Yavaş)
    # ---------------------------------------------------------
    print("1. CPU (Saf Python Loop) çalışıyor... (Biraz sürebilir)")
    start = time.time()
    # Not: Sunumda çok beklememek için N çok büyükse burayı daha küçük bir N ile çalıştırabilirsiniz
    res_loop = cpu_loop_hesapla(h_tr_d, h_tr_y, h_mat_d, h_mat_y, h_obp, N)
    time_loop = time.time() - start

    print(f"\n1. CPU (Saf Python Loop) Süresi: {time_loop:.6f} saniye\n")
    print("2. CPU (NumPy) çalışıyor... (Biraz sürebilir)")
    start = time.time()
    res_numpy = cpu_numpy_hesapla(h_tr_d, h_tr_y, h_mat_d, h_mat_y, h_obp)
    time_numpy = time.time() - start

    print(f"\n2. CPU (NumPy) Süresi: {time_numpy:.6f} saniye\n")
    print("3. GPU (CUDA) çalışıyor... (Biraz sürebilir)")
    start = time.time()
    # GPU için verileri GPU belleğine taşımak
    d_tr_d = cuda.to_device(h_tr_d)
    d_tr_y = cuda.to_device(h_tr_y)
    d_mat_d = cuda.to_device(h_mat_d)
    d_mat_y = cuda.to_device(h_mat_y)
    d_obp = cuda.to_device(h_obp)
    d_sonuclar = cuda.device_array(N, dtype=np.float32)
    
    # CUDA kernel'ı başlatma
    threadsperblock = 256
    blockspergrid = (N + (threadsperblock - 1)) // threadsperblock
    osym_cuda_kernel[blockspergrid, threadsperblock](d_tr_d, d_tr_y, d_mat_d, d_mat_y, d_obp, d_sonuclar)
    
    # Sonuçları CPU belleğine taşımak
    res_gpu = d_sonuclar.copy_to_host()
    time_gpu = time.time() - start

    print(f"\n3. GPU (CUDA) Süresi: {time_gpu:.6f} saniye\n")

    # Sonuçları kontrol etme
    assert np.allclose(res_loop, res_numpy)
    assert np.allclose(res_numpy, res_gpu)

    # Sonuçları yazdırma
    print("\nSonuçlar:")
    print("Aday 1: Loop: {:.4f}, NumPy: {:.4f}, GPU: {:.4f}".format(res_loop[0], res_numpy[0], res_gpu[0]))
    print("Aday 2: Loop: {:.4f}, NumPy: {:.4f}, GPU: {:.4f}".format(res_loop[1], res_numpy[1], res_gpu[1]))
    print("Aday 3: Loop: {:.4f}, NumPy: {:.4f}, GPU: {:.4f}".format(res_loop[2], res_numpy[2], res_gpu[2]))

    # Performans karşılaştırması
    print("\nPerformans Karşılaştırması:")
    print("CPU (Saf Python Loop): {:.2f} saniye".format(time_loop))
    print("CPU (NumPy): {:.2f} saniye".format(time_numpy))
    print("GPU (CUDA): {:.2f} saniye".format(time_gpu))

    # GPU performansını kontrol etme
    if time_gpu < time_loop:
        print("\nGPU performansı başarılı!")
    else:
        print("\nGPU performansı başarısız!")
    print("\n")
    print(f"{time_loop / time_gpu:.2f} kat daha hızlı!")

In [17]:
main()

--- 10000000 ADAY İÇİN PERFORMANS TESTİ BAŞLIYOR ---

1. CPU (Saf Python Loop) çalışıyor... (Biraz sürebilir)

1. CPU (Saf Python Loop) Süresi: 10.838080 saniye

2. CPU (NumPy) çalışıyor... (Biraz sürebilir)

2. CPU (NumPy) Süresi: 0.133962 saniye

3. GPU (CUDA) çalışıyor... (Biraz sürebilir)

3. GPU (CUDA) Süresi: 0.132178 saniye


Sonuçlar:
Aday 1: Loop: 167.3500, NumPy: 167.3500, GPU: 167.3500
Aday 2: Loop: 313.0750, NumPy: 313.0750, GPU: 313.0750
Aday 3: Loop: 339.4750, NumPy: 339.4750, GPU: 339.4750

Performans Karşılaştırması:
CPU (Saf Python Loop): 10.84 saniye
CPU (NumPy): 0.13 saniye
GPU (CUDA): 0.13 saniye

GPU performansı başarılı!


82.00 kat daha hızlı!
